# **Launch Sites Locations Analysis with Folium**


## OBJECTIVES


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.

In [66]:
import folium
import pandas as pd
import numpy as np
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

In [67]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

spacex_df=pd.read_csv(URL)

In [68]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [69]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [70]:
# nasa_coordinate = [29.559684888503615, -95.0830971930759]
# site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# # Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
# circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# # Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
# marker = folium.map.Marker(
#     nasa_coordinate,
#     # Create an icon as a text label
#     icon=DivIcon(
#         icon_size=(20,20),
#         icon_anchor=(0,0),
#         html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
#         )
#     )
# site_map.add_child(circle)
# site_map.add_child(marker)

In [71]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Define launch site names and coordinates in a dictionary
launch_sites = {
    'CCAFS LC-40': [launch_sites_df.iloc[0, 1], launch_sites_df.iloc[0, 2]],
    'CCAFS SLC-40': [launch_sites_df.iloc[1, 1], launch_sites_df.iloc[1, 2]],
    'KSC LC-39A': [launch_sites_df.iloc[2, 1], launch_sites_df.iloc[2, 2]],
    'VAFB SLC-4E': [launch_sites_df.iloc[3, 1], launch_sites_df.iloc[3, 2]]
}

# Loop through the launch sites to create circles and markers
for site_name, coordinates in launch_sites.items():
    # Add Circle
    folium.Circle(
        location=coordinates,
        radius=1000,
        color='#d35400',
        fill=True,
        popup=folium.Popup(site_name)
    ).add_to(site_map)

    # Add Marker
    folium.Marker(
        location=coordinates,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

# Display the map
site_map

# Mark the success/failed launches for each site on the map

In [72]:
marker_cluster = MarkerCluster()

In [73]:
spacex_df['marker_color'] = np.where(spacex_df['class'] == 1, 'green', 'red')

In [74]:
site_map.add_child(marker_cluster)


In [75]:
for index, record in spacex_df.iterrows():
    
    marker = folium.Marker(location= [record['Lat'], record['Long']],
                           icon = folium.Icon(color = 'white', icon_color=record['marker_color']))
    
    marker_cluster.add_child(marker)
    

site_map

In [76]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map